In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
data = [("me gusta comer en la cafeteria".split(), "SPANISH"),
        ("Give it to me".split(), "ENGLISH"),
        ("No creo que sea una buena idea".split(), "SPANISH"),
        ("No it is not a good idea to get lost at sea".split(), "ENGLISH")]

test_data = [("Yo creo que si".split(), "SPANISH"),
             ("it is lost on me".split(), "ENGLISH")]

In [5]:
word_to_idx = {}
for sent, _ in data + test_data:
    for word in sent:
        if word not in word_to_idx:
            word_to_idx[word] = len(word_to_idx)
VOCAB_SIZE = len(word_to_idx)
NUM_LABELS = 2 

In [7]:
class BoWClassifier(nn.Module):
    
    def __init__(self, num_labels, vocab_size):
        super(BoWClassifier, self).__init__()
        self.linear = nn.Linear(vocab_size, num_labels)
        
    def forward(self, bow_vec):
        return F.log_softmax(self.linear(bow_vec))
    
def make_bow_vector(sentence, word_to_idx):
    vec = torch.zeros(VOCAB_SIZE)
    for word in sentence:
        vec[word_to_idx[word]] += 1
    return vec.view(1, -1)

def make_target(label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])

In [8]:
model = BoWClassifier(NUM_LABELS, VOCAB_SIZE)

In [13]:
#for param in model.parameters():
#    print param
sample = data[0]
bow_vector = make_bow_vector(sample[0], word_to_idx)

In [16]:
log_probs = model(autograd.Variable(bow_vector))

In [20]:
label_to_ix = {'SPANISH': 0, "ENGLISH": 1}

In [22]:
for instance, label in test_data:
    bow_vec = autograd.Variable(make_bow_vector(instance, word_to_idx))
    log_probs = model(bow_vec)
    print log_probs

Variable containing:
-1.2611 -0.3332
[torch.FloatTensor of size 1x2]

Variable containing:
-1.1140 -0.3978
[torch.FloatTensor of size 1x2]



In [24]:
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [25]:
for epoch in xrange(100):
    for instance, label in data:
        model.zero_grad()
        bow_vec = autograd.Variable(make_bow_vector(instance, word_to_idx))
        log_probs = model(bow_vec)
        target = autograd.Variable(make_target(label, label_to_ix))
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()

In [26]:
for instance, label in test_data:
    bow_vec = autograd.Variable(make_bow_vector(instance, word_to_idx))
    log_probs = model(bow_vec)
    print log_probs

Variable containing:
-0.2209 -1.6186
[torch.FloatTensor of size 1x2]

Variable containing:
-3.0979 -0.0462
[torch.FloatTensor of size 1x2]

